# Notebook 1: Basic PSD Workflow

## Introduction to Pulse Shape Discrimination (PSD)

### What is PSD?

Pulse Shape Discrimination (PSD) is a powerful technique used in radiation detection to distinguish between different types of particles (primarily neutrons and gamma rays) based on the temporal characteristics of scintillation light pulses.

### Physical Principles

When ionizing radiation interacts with organic scintillators, energy is deposited through:
- **Gamma rays**: Compton scattering, photoelectric effect, pair production
- **Neutrons**: Elastic scattering with hydrogen nuclei (proton recoils)

The scintillation light consists of two components:
1. **Fast component** (singlet states): ~3-5 ns decay time
2. **Slow component** (triplet states): ~30-50 ns decay time

**Key insight**: The ratio of slow-to-fast light differs between particle types!
- Neutrons (heavy charged recoils): Higher slow/fast ratio → longer tail
- Gammas (light charged recoils): Lower slow/fast ratio → shorter tail

### The Tail-to-Total Method

The most common PSD method is the **charge integration technique**:

$$\text{PSD} = \frac{Q_{\text{tail}}}{Q_{\text{total}}} = \frac{Q_{\text{long}} - Q_{\text{short}}}{Q_{\text{long}}}$$

where:
- $Q_{\text{short}}$: Integrated charge in fast gate (~0-200 ns)
- $Q_{\text{long}}$: Integrated charge in long gate (~0-800 ns)
- $Q_{\text{tail}}$: Tail component

### Learning Objectives

In this notebook, you will learn to:
1. Generate synthetic PSD waveforms
2. Calculate the PSD parameter using tail-to-total ratio
3. Visualize neutron vs gamma event distributions
4. Understand the Figure of Merit (FoM)
5. Perform basic particle discrimination

## Setup and Imports

In [ ]:
# Standard scientific computing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import norm

# Set plotting style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Ensure reproducibility
np.random.seed(42)

print("Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

## 1. Synthetic Waveform Generation

### Scintillation Pulse Model

A scintillation pulse can be modeled as a bi-exponential function:

$$V(t) = A \left[ f_{\text{fast}} \cdot e^{-t/\tau_{\text{fast}}} + (1-f_{\text{fast}}) \cdot e^{-t/\tau_{\text{slow}}} \right]$$

where:
- $A$: Pulse amplitude (proportional to energy)
- $f_{\text{fast}}$: Fast component fraction
- $\tau_{\text{fast}}$: Fast decay constant (~3 ns for EJ-301)
- $\tau_{\text{slow}}$: Slow decay constant (~30 ns for EJ-301)

**Critical difference**:
- Gamma rays: $f_{\text{fast}} \approx 0.75$ (more fast light)
- Neutrons: $f_{\text{fast}} \approx 0.55$ (more slow light)

In [ ]:
def generate_scintillation_pulse(particle_type, energy_kev, 
                                 sampling_rate_mhz=250, 
                                 num_samples=368):
    """
    Generate synthetic scintillation waveform
    
    Parameters:
    -----------
    particle_type : str
        'neutron' or 'gamma'
    energy_kev : float
        Particle energy in keV electron-equivalent
    sampling_rate_mhz : float
        Digitizer sampling rate (default 250 MHz = 4 ns/sample)
    num_samples : int
        Number of waveform samples
    
    Returns:
    --------
    waveform : array
        ADC values (baseline + negative pulse)
    """
    # Time array (in nanoseconds)
    dt = 1000.0 / sampling_rate_mhz  # ns per sample
    time = np.arange(num_samples) * dt
    
    # Scintillator properties (EJ-301 liquid scintillator)
    tau_fast = 3.2  # ns
    tau_slow = 32.0  # ns
    
    # Particle-dependent fast fraction
    if particle_type == 'gamma':
        fast_fraction = 0.75  # 75% fast light
    elif particle_type == 'neutron':
        fast_fraction = 0.55  # 55% fast light (more tail!)
    else:
        raise ValueError("particle_type must be 'neutron' or 'gamma'")
    
    # Pulse amplitude (proportional to energy)
    # Scale to typical ADC values (14-bit: 0-16383)
    amplitude = energy_kev * 3.0  # ~3 ADC counts per keV
    
    # Pulse start time (after baseline region)
    t0 = 200  # ns (50 samples at 250 MHz)
    
    # Bi-exponential scintillation pulse
    pulse = np.zeros_like(time)
    active_time = time - t0
    valid = active_time >= 0
    
    pulse[valid] = amplitude * (
        fast_fraction * np.exp(-active_time[valid] / tau_fast) +
        (1 - fast_fraction) * np.exp(-active_time[valid] / tau_slow)
    )
    
    # Baseline (inverted: ADC measures negative-going pulses)
    baseline = 8192  # Mid-range for 14-bit ADC
    waveform = baseline - pulse  # Negative-going pulse
    
    # Add realistic noise (Gaussian, ~10 ADC counts RMS)
    noise = np.random.normal(0, 10, num_samples)
    waveform += noise
    
    # Ensure ADC range
    waveform = np.clip(waveform, 0, 16383)
    
    return waveform.astype(int)

# Generate example pulses
gamma_pulse = generate_scintillation_pulse('gamma', energy_kev=500)
neutron_pulse = generate_scintillation_pulse('neutron', energy_kev=500)

print("✓ Waveform generation function defined")
print(f"  Gamma pulse shape: {gamma_pulse.shape}")
print(f"  Neutron pulse shape: {neutron_pulse.shape}")

## 2. Visualize Neutron vs Gamma Pulses

Let's compare the pulse shapes side-by-side to understand the physical difference.

In [ ]:
# Generate multiple pulses for averaging
n_pulses = 100
sampling_rate = 250  # MHz
dt = 1000.0 / sampling_rate  # ns per sample

gamma_pulses = []
neutron_pulses = []

for _ in range(n_pulses):
    energy = np.random.uniform(400, 600)  # keV
    gamma_pulses.append(generate_scintillation_pulse('gamma', energy))
    neutron_pulses.append(generate_scintillation_pulse('neutron', energy))

# Average and normalize
gamma_avg = np.mean(gamma_pulses, axis=0)
neutron_avg = np.mean(neutron_pulses, axis=0)

# Baseline subtract and normalize
baseline = np.mean(gamma_avg[:50])
gamma_norm = (baseline - gamma_avg) / np.max(baseline - gamma_avg)
neutron_norm = (baseline - neutron_avg) / np.max(baseline - neutron_avg)

# Time axis
time_ns = np.arange(len(gamma_norm)) * dt

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Full pulse comparison
ax = axes[0]
ax.plot(time_ns, gamma_norm, 'b-', linewidth=2.5, label='Gamma', alpha=0.8)
ax.plot(time_ns, neutron_norm, 'r-', linewidth=2.5, label='Neutron', alpha=0.8)
ax.set_xlabel('Time (ns)', fontsize=13, fontweight='bold')
ax.set_ylabel('Normalized Amplitude', fontsize=13, fontweight='bold')
ax.set_title('Scintillation Pulse Shapes (Average of 100 events)', 
             fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim(150, 600)

# Tail region (zoomed)
ax = axes[1]
ax.plot(time_ns, gamma_norm, 'b-', linewidth=2.5, label='Gamma (shorter tail)', alpha=0.8)
ax.plot(time_ns, neutron_norm, 'r-', linewidth=2.5, label='Neutron (longer tail)', alpha=0.8)
ax.set_xlabel('Time (ns)', fontsize=13, fontweight='bold')
ax.set_ylabel('Normalized Amplitude (log scale)', fontsize=13, fontweight='bold')
ax.set_title('Tail Region - Key PSD Signature', fontsize=14, fontweight='bold')
ax.legend(fontsize=12, loc='upper right')
ax.set_yscale('log')
ax.grid(True, alpha=0.3, which='both')
ax.set_xlim(250, 600)
ax.set_ylim(0.01, 1.2)

# Add annotations
ax.annotate('Neutrons have\nmore slow component', 
            xy=(400, 0.15), xytext=(450, 0.4),
            arrowprops=dict(arrowstyle='->', color='red', lw=2),
            fontsize=11, color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ Pulse shape comparison plotted")
print("\nKey observation:")
print("  Neutron pulses have a LONGER TAIL due to higher triplet state contribution")

## 3. Calculate PSD Parameter

### Charge Integration Method

We integrate the pulse in two time windows:
- **Short gate**: 0-200 ns (captures mostly fast component)
- **Long gate**: 0-800 ns (captures fast + slow components)

The PSD parameter quantifies the tail fraction.

In [ ]:
def calculate_psd_parameter(waveform, short_gate_ns=200, long_gate_ns=800, 
                           sampling_rate_mhz=250, baseline_samples=50):
    """
    Calculate PSD parameter using charge integration
    
    Parameters:
    -----------
    waveform : array
        Raw ADC waveform
    short_gate_ns : float
        Short integration window (ns)
    long_gate_ns : float
        Long integration window (ns)
    sampling_rate_mhz : float
        Digitizer sampling rate
    baseline_samples : int
        Number of pre-trigger samples for baseline
    
    Returns:
    --------
    psd : float
        PSD parameter (0-1, higher = more tail)
    energy : float
        Total integrated charge (long gate)
    energy_short : float
        Short gate integrated charge
    """
    dt = 1000.0 / sampling_rate_mhz  # ns per sample
    
    # Baseline subtraction
    baseline = np.mean(waveform[:baseline_samples])
    pulse = baseline - waveform  # Convert to positive-going
    pulse[pulse < 0] = 0  # Remove negative excursions (noise)
    
    # Find pulse start (first crossing above threshold)
    threshold = 0.05 * np.max(pulse)
    start_idx = np.where(pulse > threshold)[0]
    if len(start_idx) == 0:
        return 0, 0, 0
    
    start_idx = start_idx[0]
    
    # Convert gate times to sample indices
    short_samples = int(short_gate_ns / dt)
    long_samples = int(long_gate_ns / dt)
    
    # Integration windows (from pulse start)
    short_end = min(start_idx + short_samples, len(pulse))
    long_end = min(start_idx + long_samples, len(pulse))
    
    # Integrate charge
    Q_short = np.sum(pulse[start_idx:short_end])
    Q_long = np.sum(pulse[start_idx:long_end])
    
    # PSD parameter
    if Q_long > 0:
        psd = (Q_long - Q_short) / Q_long
    else:
        psd = 0
    
    return psd, Q_long, Q_short

# Test on example pulses
psd_gamma, E_gamma, _ = calculate_psd_parameter(gamma_pulse)
psd_neutron, E_neutron, _ = calculate_psd_parameter(neutron_pulse)

print("✓ PSD calculation function defined")
print(f"\nExample calculations:")
print(f"  Gamma:   PSD = {psd_gamma:.4f}, Energy = {E_gamma:.0f}")
print(f"  Neutron: PSD = {psd_neutron:.4f}, Energy = {E_neutron:.0f}")
print(f"\n  → Neutron PSD is higher (more tail component)")

## 4. Generate Dataset and Visualize PSD Distribution

Now let's create a realistic dataset with both neutrons and gammas across an energy range.

In [ ]:
# Generate synthetic dataset
n_events = 5000  # Events per particle type

data = []

print("Generating synthetic PSD dataset...")

for particle in ['gamma', 'neutron']:
    for i in range(n_events):
        # Random energy (50-2000 keV electron-equivalent)
        energy_kev = np.random.exponential(400) + 50
        energy_kev = min(energy_kev, 2000)
        
        # Generate waveform
        waveform = generate_scintillation_pulse(particle, energy_kev)
        
        # Calculate PSD
        psd, energy, energy_short = calculate_psd_parameter(waveform)
        
        data.append({
            'particle': particle,
            'energy': energy,
            'energy_short': energy_short,
            'psd': psd,
            'true_energy_kev': energy_kev
        })

# Create DataFrame
df = pd.DataFrame(data)

print(f"✓ Generated {len(df)} events")
print(f"\nDataset summary:")
print(df.groupby('particle').agg({
    'psd': ['mean', 'std'],
    'energy': ['mean', 'std']
}).round(4))

## 5. PSD vs Energy Scatter Plot

The classic PSD visualization shows clear separation between neutrons and gammas.

In [ ]:
# Create PSD scatter plot
fig, ax = plt.subplots(figsize=(12, 8))

# Plot gamma events
gamma_data = df[df['particle'] == 'gamma']
ax.scatter(gamma_data['energy'], gamma_data['psd'], 
           c='blue', alpha=0.3, s=10, label='Gamma rays', edgecolors='none')

# Plot neutron events
neutron_data = df[df['particle'] == 'neutron']
ax.scatter(neutron_data['energy'], neutron_data['psd'], 
           c='red', alpha=0.3, s=10, label='Neutrons', edgecolors='none')

ax.set_xlabel('Energy (ADC integrated charge)', fontsize=13, fontweight='bold')
ax.set_ylabel('PSD Parameter', fontsize=13, fontweight='bold')
ax.set_title('Pulse Shape Discrimination: Neutron vs Gamma Separation', 
             fontsize=15, fontweight='bold')
ax.legend(fontsize=12, markerscale=3)
ax.grid(True, alpha=0.3)
ax.set_xlim(0, df['energy'].quantile(0.99))
ax.set_ylim(0, 0.5)

# Add separation annotations
ax.axhline(0.25, color='green', linestyle='--', linewidth=2, alpha=0.7)
ax.text(df['energy'].max()*0.7, 0.26, 'Discrimination threshold', 
        fontsize=11, color='green', fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ PSD scatter plot created")
print("\nKey insight: Clear separation between particle types!")
print("  Gammas cluster at lower PSD values (~0.15-0.25)")
print("  Neutrons cluster at higher PSD values (~0.30-0.40)")

## 6. Figure of Merit (FoM)

### Definition

The Figure of Merit quantifies the quality of n/γ separation:

$$\text{FoM} = \frac{|\mu_n - \mu_\gamma|}{\text{FWHM}_n + \text{FWHM}_\gamma}$$

where:
- $\mu_n, \mu_\gamma$: Mean PSD values
- $\text{FWHM}$: Full Width at Half Maximum (2.355 × σ for Gaussian)

**Quality criteria**:
- FoM > 1.5: Excellent discrimination
- FoM > 1.0: Good discrimination  
- FoM > 0.5: Moderate discrimination
- FoM < 0.5: Poor discrimination

In [ ]:
def calculate_figure_of_merit(psd_neutron, psd_gamma):
    """
    Calculate Figure of Merit for n/γ separation
    
    Parameters:
    -----------
    psd_neutron : array
        PSD values for neutron events
    psd_gamma : array
        PSD values for gamma events
    
    Returns:
    --------
    fom : float
        Figure of merit
    """
    # Calculate means
    mean_n = np.mean(psd_neutron)
    mean_g = np.mean(psd_gamma)
    separation = abs(mean_n - mean_g)
    
    # Calculate FWHM (2.355 * sigma for Gaussian)
    fwhm_n = 2.355 * np.std(psd_neutron)
    fwhm_g = 2.355 * np.std(psd_gamma)
    
    # Figure of Merit
    fom = separation / (fwhm_n + fwhm_g)
    
    return fom, mean_n, mean_g, fwhm_n, fwhm_g

# Calculate FoM for energy slice (500-700 keV)
energy_slice = (df['energy'] > 1200) & (df['energy'] < 1800)
neutron_psd = df[(df['particle'] == 'neutron') & energy_slice]['psd'].values
gamma_psd = df[(df['particle'] == 'gamma') & energy_slice]['psd'].values

fom, mu_n, mu_g, fwhm_n, fwhm_g = calculate_figure_of_merit(neutron_psd, gamma_psd)

print("✓ Figure of Merit calculated")
print(f"\nSeparation quality (energy slice 1200-1800):")
print(f"  Neutron: μ = {mu_n:.4f}, FWHM = {fwhm_n:.4f}")
print(f"  Gamma:   μ = {mu_g:.4f}, FWHM = {fwhm_g:.4f}")
print(f"  Figure of Merit = {fom:.3f}")

if fom > 1.5:
    quality = "Excellent"
elif fom > 1.0:
    quality = "Good"
elif fom > 0.5:
    quality = "Moderate"
else:
    quality = "Poor"

print(f"  Quality: {quality} discrimination")

## 7. PSD Distributions with FoM Visualization

In [ ]:
# Plot PSD histograms
fig, ax = plt.subplots(figsize=(12, 7))

# Histograms
bins = np.linspace(0, 0.5, 80)
ax.hist(gamma_psd, bins=bins, alpha=0.6, label='Gamma', 
        color='blue', edgecolor='black', linewidth=1.2)
ax.hist(neutron_psd, bins=bins, alpha=0.6, label='Neutron', 
        color='red', edgecolor='black', linewidth=1.2)

# Mark means
ax.axvline(mu_g, color='blue', linestyle='--', linewidth=2.5, 
           label=f'Gamma mean: {mu_g:.3f}')
ax.axvline(mu_n, color='red', linestyle='--', linewidth=2.5, 
           label=f'Neutron mean: {mu_n:.3f}')

# Mark FWHM regions
ax.axvspan(mu_g - fwhm_g/2, mu_g + fwhm_g/2, alpha=0.15, color='blue')
ax.axvspan(mu_n - fwhm_n/2, mu_n + fwhm_n/2, alpha=0.15, color='red')

# Labels and formatting
ax.set_xlabel('PSD Parameter', fontsize=13, fontweight='bold')
ax.set_ylabel('Counts', fontsize=13, fontweight='bold')
ax.set_title(f'PSD Distributions - Figure of Merit = {fom:.3f} ({quality})', 
             fontsize=15, fontweight='bold')
ax.legend(fontsize=11, loc='upper right')
ax.grid(True, alpha=0.3, axis='y')

# Add FoM annotation
ax.annotate('', xy=(mu_g, ax.get_ylim()[1]*0.8), 
            xytext=(mu_n, ax.get_ylim()[1]*0.8),
            arrowprops=dict(arrowstyle='<->', color='green', lw=3))
ax.text((mu_g + mu_n)/2, ax.get_ylim()[1]*0.85, 'Separation', 
        ha='center', fontsize=12, color='green', fontweight='bold')

plt.tight_layout()
plt.show()

print("✓ PSD distribution plotted with FoM visualization")

## 8. Simple Linear Discrimination

Apply a simple threshold to classify events.

In [ ]:
# Define discrimination threshold (midpoint between means)
threshold = (mu_n + mu_g) / 2

# Apply classification
df['predicted'] = df['psd'].apply(lambda x: 'neutron' if x > threshold else 'gamma')

# Calculate accuracy
correct = (df['particle'] == df['predicted']).sum()
accuracy = correct / len(df) * 100

# Confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(df['particle'], df['predicted'], labels=['gamma', 'neutron'])

print("✓ Simple discrimination applied")
print(f"\nDiscrimination threshold: PSD = {threshold:.4f}")
print(f"Overall accuracy: {accuracy:.2f}%")
print("\nConfusion Matrix:")
print("                 Predicted")
print("                 Gamma  Neutron")
print(f"Actual Gamma     {cm[0,0]:5d}  {cm[0,1]:5d}")
print(f"Actual Neutron   {cm[1,0]:5d}  {cm[1,1]:5d}")

print("\nClassification Report:")
print(classification_report(df['particle'], df['predicted'], 
                          target_names=['Gamma', 'Neutron']))

## Summary and Key Takeaways

### What We Learned

1. **Physical Principle**: Neutrons and gammas produce different pulse shapes due to different ionization densities
   - Neutrons → more slow scintillation light (triplet states)
   - Gammas → more fast scintillation light (singlet states)

2. **PSD Parameter**: The tail-to-total ratio effectively quantifies this difference
   - Simple to calculate (just two charge integrations)
   - Hardware-implementable (FPGA charge integrators)

3. **Figure of Merit**: Quantifies discrimination quality
   - FoM > 1.0 indicates good separation
   - Energy-dependent (typically measured at ~1 MeVee)

4. **Discrimination Performance**: Even a simple threshold achieves high accuracy
   - Typical accuracy: >95% for good scintillators
   - Energy threshold: PSD quality degrades below ~200 keVee

### Next Steps

In the following notebooks, we will explore:
- **Notebook 2**: Energy calibration to convert ADC → keV
- **Notebook 3**: Advanced feature extraction (100+ timing features)
- **Notebook 4**: Machine learning for improved discrimination
- **Notebook 5**: Isotope identification from gamma spectra
- **Notebook 6**: Deep learning with raw waveforms
- **Notebook 7**: Scintillator characterization
- **Notebook 8**: Advanced techniques (real-time, FPGA, physics-informed ML)

### References

1. Brooks, F. D. (1979). "Development of organic scintillators." *Nuclear Instruments and Methods*, 162(1-3), 477-505.
2. Knoll, G. F. (2010). *Radiation Detection and Measurement* (4th ed.). Wiley.
3. Marrone, S., et al. (2002). "Pulse shape analysis of signals from BaF2 and CeF3 scintillators for neutron/γ-ray discrimination." *Nuclear Instruments and Methods A*, 490(1-2), 299-307.